In [ ]:
#using google cloud example https://cloud.google.com/blog/big-data/2017/10/intro-to-text-classification-with-keras-automatically-tagging-stack-overflow-posts
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.layers import Conv1D, Dense, Flatten, Input
from keras.models import Sequential

In [2]:
train = pd.read_csv("~/.kaggle/competitions/spooky-author-identification/train.csv")
test = pd.read_csv("~/.kaggle/competitions/spooky-author-identification/test.csv")
sample_submission = pd.read_csv("~/.kaggle/competitions/spooky-author-identification/sample_submission.csv")

In [3]:
train_size = int(len(train) * .8)
train_words = train["text"][:train_size]
train_target = train["author"][:train_size]
test_words = train["text"][train_size:]
test_target = train["author"][train_size:]

In [4]:
vocab_size = 1000
num_labels=3
tokenize = Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(train_words)

x_train = tokenize.texts_to_matrix(train_words)

In [5]:
encoder = LabelBinarizer()
encoder.fit(train_words)
y_train = encoder.transform(train_target)
y_test = encoder.transform(test_target)
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [6]:
model = Sequential()
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

ValueError: Input 0 is incompatible with layer flatten_1: expected min_ndim=3, found ndim=2

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.fit(x_train, y_train, 
        batch_size=5, 
        epochs=2, 
        validation_split=0.1)

In [ ]:
score = model.evaluate(x_test, y_test, 
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])